# Practica de imputación


## 0. importar librerias

In [1]:
#Importamos las librerias que usaremos en este practica

import pandas as pd
import numpy as np
import seaborn as sns

sns.set_theme(style="whitegrid")

## 1. Cargar el data set

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/diabetes-NAN.csv',    #Ruta completa a la ubicación del archivo de interés.
                 na_values='?')              #Como el archivo contiene algunos caracteres ?, con este comando pandas los interpreta como nan (not a number).

## 2. Observaciones basicas del dataset

### Tamaño del dataset

In [3]:
df.shape

(768, 9)

### información general

Podemos observar el nombre de las columnas, si tienen nulos y cual es su tipo de dato

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               766 non-null    float64
 1   Glucose                   759 non-null    float64
 2   BloodPressure             761 non-null    float64
 3   SkinThickness             759 non-null    float64
 4   Insulin                   762 non-null    float64
 5   BMI                       761 non-null    float64
 6   DiabetesPedigreeFunction  766 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 54.1 KB


Gracias a esta información, podemos ver que tenemos una pequeña cantidad de datos nulos

### Vista preliminar

Observamos los primeros y ultimos 5 datos del data set

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50,1
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31,0
2,8.0,NaN,64.0,0.0,0.0,23.3,0.672,32,1
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0.0,NaN,NaN,35.0,168.0,43.1,2.288,33,1


In [6]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27,0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47,1
767,1.0,93.0,70.0,31.0,0.0,30.4,0.315,23,0


datos nulos

In [7]:
df.isnull().sum()

,0
Pregnancies,2
Glucose,9
BloodPressure,7
SkinThickness,9
Insulin,6
BMI,7
DiabetesPedigreeFunction,2
Age,0
Outcome,0


# 3 Imputación de datos nulos

La imputación consiste en darle valor a nuestros datos nulos para recuperar la información perdida, observaremos dos metodos en esta practica, usando una versión de knn y con la media.

### 3.1 Uso de la media

Cuando tenemos datos nulos, podemos reemplazarlos con la media de la columna (Si es categorica usamos la moda)

In [8]:
#Gracias a pandas tenemos un metodo que hace esto mucho más simple
#Le aplicamos a una columna del df (en este caso glucose) el metodo fillna
#En los parametros de fillna le damos la media de la columna y el inplace=true (el inplace true significa que va modificar el df)
df['Glucose'].fillna(df['Glucose'].mean(), inplace=True)

In [9]:
df.isnull().sum()

,0
Pregnancies,2
Glucose,0
BloodPressure,7
SkinThickness,9
Insulin,6
BMI,7
DiabetesPedigreeFunction,2
Age,0
Outcome,0


como podemos ver, ahora no tenemos nulos en la columna 'Glucose'

### 3.2 KNNImputer

En este caso usaremos un modelo de knn para darle un valor a nuestros valores nulos.

In [10]:
from sklearn.impute import KNNImputer

# Construimos el modelo
imputer = KNNImputer(n_neighbors=5, weights="uniform")

# Entrenamos el modelo
imputer.fit(df[["SkinThickness"]])

#Reemplazamos los valores nulos
df["SkinThickness"] = imputer.transform(df[["SkinThickness"]]).ravel()

In [11]:
df.isnull().sum()

,0
Pregnancies,2
Glucose,0
BloodPressure,7
SkinThickness,0
Insulin,6
BMI,7
DiabetesPedigreeFunction,2
Age,0
Outcome,0


## 4 Comparación borrar nulos vs imputación

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/diabetes-NAN.csv', na_values='?')

### 4.1 borrando los nulos

In [13]:
df_clean=df.dropna()

In [14]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables predictoras
y= df_clean[objetivo].values
X= df_clean[predictores].values

In [15]:
from sklearn.model_selection import train_test_split
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(595, 8)
(149, 8)
(595,)
(149,)


In [16]:
# Se importa el modelo
from sklearn.linear_model import LogisticRegression

#Se crea el modelo
log_reg = LogisticRegression(solver='lbfgs',max_iter=1000)
#se entrena el modelo
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [17]:
print("resultado borrando los nulos")
log_reg.score(X_test, y_test)

resultado borrando los nulos


0.8187919463087249

### 4.2 Reemplazando los nulos con la media

In [18]:
df.isnull().sum()

,0
Pregnancies,2
Glucose,9
BloodPressure,7
SkinThickness,9
Insulin,6
BMI,7
DiabetesPedigreeFunction,2
Age,0
Outcome,0


In [19]:
df_mean =df[predictores].fillna(df[predictores].mean())

In [20]:
df_mean.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0


In [21]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables que usaremos para encontrar a Y
y= df[objetivo].values
X= df_mean[predictores].values

In [22]:
from sklearn.model_selection import train_test_split
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [23]:
# Se importa el modelo
from sklearn.linear_model import LogisticRegression

#Se crea el modelo
log_reg = LogisticRegression(solver='lbfgs',max_iter=1000)
#se entrena el modelo
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [24]:
print("resultado con imputación por la media")
log_reg.score(X_test, y_test)

resultado con imputación por la media


0.7467532467532467

### 4.3 Reemplazando los nulos con knn

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/diabetes-NAN.csv', na_values='?')

In [26]:
# Construimos el modelo
imputer = KNNImputer(n_neighbors=5, weights="uniform")

#Creamos un nuevo dataframe
df_knnI= df.copy()

#Creamos un loop para recorrer todas las columnas que queremos modificar

for column in predictores:

  # Entrenamos el modelo
  imputer.fit(df[[column]])
  #Reemplazamos los valores nulos
  df_knnI[column] = imputer.transform(df[[column]]).ravel()

In [27]:
df_knnI

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.000000,72.000000,35.0,0.0,33.6,0.627,50.0,1
1,1.0,85.000000,66.000000,29.0,0.0,26.6,0.351,31.0,0
2,8.0,120.603426,64.000000,0.0,0.0,23.3,0.672,32.0,1
3,1.0,89.000000,66.000000,23.0,94.0,28.1,0.167,21.0,0
4,0.0,120.603426,69.147175,35.0,168.0,43.1,2.288,33.0,1
...,...,...,...,...,...,...,...,...,...
763,10.0,101.000000,76.000000,48.0,180.0,32.9,0.171,63.0,0
764,2.0,122.000000,70.000000,27.0,0.0,36.8,0.340,27.0,0
765,5.0,121.000000,72.000000,23.0,112.0,26.2,0.245,30.0,0
766,1.0,126.000000,60.000000,0.0,0.0,30.1,0.349,47.0,1


In [28]:
df_knnI.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [29]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables que usaremos para encontrar a Y
y= df[objetivo].values
X= df_knnI[predictores].values

In [30]:
from sklearn.model_selection import train_test_split
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [31]:
# Se importa el modelo
from sklearn.linear_model import LogisticRegression

#Se crea el modelo
log_reg = LogisticRegression(solver='lbfgs',max_iter=1000)
#se entrena el modelo
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [32]:
print("resultado con imputación con KNN")
log_reg.score(X_test, y_test)

resultado con imputación con KNN


0.7467532467532467

### 4.3 Con nulos

In [33]:
df = pd.read_csv('https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/diabetes-NAN.csv', na_values='?')

In [34]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables que usaremos para encontrar a Y
y= df[objetivo].values
X= df[predictores].values

In [35]:
from sklearn.model_selection import train_test_split
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [36]:
from sklearn.ensemble import (HistGradientBoostingClassifier)

hgbm = HistGradientBoostingClassifier(random_state=42)
hgbm.fit(X_train, y_train)
hgbm.score(X_test, y_test)

0.7272727272727273

## 5 Conclusión

Podemos ver que el mejor modelo fue el que se entreno solo borrando los nulos, esto se debe a que la cantidad de nulos en el data set no era muy alta y no se perdió mucha información.

Dicho esto, cabe resaltar que los dos modelos con la imputación fueron mejores que el modelo con nulos.

En conclusión, aunque los modelos de imputación son buenos, deberían ser usando en casos donde la cantidad de datos nulos en una sola columna sea muy alta. Si tenemos pocos nulos, como se pudo observar, la mejor opción es borrar esas observaciones.


In [ ]:
# Realice la comparación de los métodos de imputación con un dataset propio